# High-Level API

![pawns](../_static/images/banner/pawns.png)

## Declarative

The *High-Level API* is *declarative*. What does that mean? All you have to do is specify the *state* that you want the data in, and then the backend executes all of the tedious data wrangling needed to achieve that state. It's like Terraform for machine learning.

> Under the hood, the High-Level API is actually weaving together a workflow using the *object-relational model (ORM)* of the [Low-Level API](api_low_level.html). Practitioners can use the low-level interface themselves, but it's a bit more cumbersome in that you need to string together object IDs. 

1. `Pipeline` declares how to preprocess data.
2. `Experiment` declares variations of models to train and evaluate.
3. `Inference` declares new samples to predict.

Reference the [tutorials](../pages/gallery.html) to the see the high level API in action for various types of data and analysis.

```
from aiqc.mlops import *
```

---

## 1. Pipeline

Declares how to prepare data. This workflow is used during:

- Preprocessing of training and evaluation data.
- Caching of preprocessed training and evaluation data.
- Post-processing (e.g. decoding) during model evaluation.
- Inference: encoding and decoding new data.

```python
Pipeline(
    inputs:list
    , target:object     = None
    , stratifier:object = None
    , name:str          = None
    , description:str   = None
)
```

| Argument | Description |
| --- | --- |
| *inputs* | Reference `Input` below. Multiple inputs may be provided for heterogenous/ multi-modal analysis. |
| *target* | Reference `Target` below. Left blank during unsupervised/ self-supervised analysis. |
| *stratifier* | Reference `Stratifier` below. Left blank during inference. |
| *name* | An auto-incrementing version will be assigned to `Pipelines` that share a name.  |
| *description* | Describes how this particular workflow is unique. |

> It is possible for an `Input` and a `Target` to share a `dataset`. The `Input.include_columns` and `Input.exclude_columns` will automatically be adjusted to exclude `Target.column`.

| Returns |
| --- |
| A `Splitset` object as seen in the [Low-Level API](api_low_level.html). | 

---

### 1a. Input

These are the features that our model will learn from. This is a wrapper for `Feature` in the [Low-Level API](api_low_level.html).

```python
Input(
    dataset:object
    , exclude_columns:list  = None
    , include_columns:list  = None
    , interpolaters:list    = None
    , window:object         = None
    , encoders:list         = None
    , reshape_indices:tuple = None
)
```

| Argument | Description |
| --- | --- |
| *dataset* | Reference `Dataset` in the [Low-Level API](api_low_level.html). |
| *exclude_columns* | The columns from the `Dataset` that will not be used. |
| *include_columns* | The columns from the `Dataset` that will be used. |
| *interpolaters* | Reference `Input.Interpolater` below. |
| *window* | Reference `Input.Window` below. |
| *encoders* | Reference `Input.Encoder` below. |
| *reshape_indices* | *nadarray.shape* indices used to reshape the final feature array. Reference `FeatureShaper` in the [Low-Level API](api_low_level.html).  |

> Both `exclude_columns` and `include_columns` cannot be used simultaneously.


#### 1ai. Input.Interpolater

Used to fill in the blanks in a sequence. This is a wrapper for `FeatureInterpolater` in the [Low-Level API](api_low_level.html).

```python
Input.Interpolater(
    process_separately:bool   = True
    , verbose:bool            = False
    , interpolate_kwargs:dict = None
    , dtypes:list             = None
    , columns:list            = None
)
```

#### 1aii. Input.Window

Used to slice and shift samples into many time series windows for walk-forward/ backward analysis. This is a wrapper for `Window` in the [Low-Level API](api_low_level.html).

```
Input.Window(size_window:int, size_shift:int)
```

#### 1aiii. Input.Encoder

Used to numerically encode data. This is a wrapper for `FeatureCoder` in the [Low-Level API](api_low_level.html).

```python
Input.Encoder(
    sklearn_preprocess:object
    , verbose:bool = False
    , include:bool = True
    , dtypes:list  = None
    , columns:list = None
)
```

---

### 1b. Target

What the model is trying to predict. This is a wrapper for `Label` in the [Low-Level API](api_low_level.html).

```python
Target(
    dataset:object
    , column:str          = None
    , interpolater:object = None
    , encoder:object      = None
)
```

| Argument | Description |
| --- | --- |
| *dataset* | Reference `Dataset` in the [Low-Level API](api_low_level.html). |
| *column* | The column from the `Dataset` to use as the target. | 
| *interpolater* | Reference `Target.Interpolater` below. |
| *encoder* | Reference `Target.Encoder` below. |

#### 1bi. Target.Interpolater

Used to fill in the blanks in a sequence. This is a wrapper for `LabelInterpolater` in the [Low-Level API](api_low_level.html).

```python
Target.Interpolater(
    process_separately:bool=True
    , interpolate_kwargs:dict=None
)
```

#### 1bii. Target.Encoder

Used to numerically encode data. This is a wrapper for `LabelCoder` in the [Low-Level API](api_low_level.html). 

```
Target.Encoder(sklearn_preprocess:object)
```

---

### 1c. Stratifier

Used to slice the dataset into training, validation, test, and/or cross-validated subsets. This is a wrapper for `Splitset` in the [Low-Level API](api_low_level.html). 

```python
Stratifier(
    size_test:float         = None
    , size_validation:float = None
    , fold_count:int        = None
    , bin_count:int         = None
)
```

---

## 2. Experiment

Used to declare variations of models that will be trained. This is a wrapper for `Queue` in the [Low-Level API](api_low_level.html).

```python
Experiment(
    architecture:object
    , trainer:object
)
```

| Argument | Description |
| --- | --- |
| *architecture* | Reference `Architecture` below. |
| *trainer* | Reference `Trainer` below. |

| Returns |
| --- |
| A `Queue` object as seen in the [Low-Level API](api_low_level.html). | 

### 2a. Architecture

The model and hyperparameters to be trained. This is a wrapper for `Algorithm` in the [Low-Level API](api_low_level.html).

```python
Architecture(
    library:str
    , analysis_type:str
    , fn_build:object
    , fn_train:object
    , fn_optimize:object = None
    , fn_lose:object     = None
    , fn_predict:object  = None
    , hyperparameters:dict = None
)
```

### 2b. Trainer

The options used for training. This is a wrapper for `Queue` in the [Low-Level API](api_low_level.html).

```python
Trainer(
    pipeline_id:int
    , repeat_count:int     = 1
    , permute_count:int    = 3
    , search_count         = None
    , search_percent       = None
)
```

---

## 3. Inference

Used to preprocess new samples, run predictions on them, decode the output, and, optionally, evaluate the predictions.

```python
Inference(
    predictor:object
    , feature_datasets:list
    , label_dataset:object=None
)
```

| Argument | Description |
| --- | --- |
| *predictor* | Reference `Predictor` in the [Low-Level API](api_low_level.html). | 
| *input_datasets* | Reference `Dataset` in the [Low-Level API](api_low_level.html). |
| *target_dataset* | Leave this blank for pure inference where no metrics will be calculared. Reference `Dataset` in the [Low-Level API](api_low_level.html). |

> We don't need to specify fully-fledged `Inputs` and `Target` objects because the `Pipeline` of the `predictor` object will be reused in order to process these new datasets.

| Returns |
| --- |
| A `Prediction` object as seen in the [Low-Level API](api_low_level.html). | 